In [47]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
import ta
import matplotlib.pyplot as plt
import vnstock as vn
from datetime import timedelta
from vnstock3 import Vnstock

# Setting

In [48]:
rf = {'2019': 0.0451, '2020': 0.0286, '2021':0.023,'2022': 0.0335,'2023':0.0321}
rf = pd.Series(rf)
rf_cal = rf.mean()
rf_2022 = 0.0335

In [49]:
pd.set_option('display.max_columns', None) 

In [50]:
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
OBV_PERIOD = 5
initial_investment = 100000000


In [51]:
win_rate = 0.7070060208
loss_rate = 1 - win_rate
mean_profit = 0.1721618831
mean_loss = 0.1082770633
stop_loss = 0.08

In [52]:
def kelly_criterion(p, q, profit, loss):
    b = (profit * 160_000_000)/(loss * 160_000_000)
    f = (b*p - q)/b
    return f

In [53]:
f =kelly_criterion(win_rate,loss_rate,mean_profit,mean_loss)

In [54]:
high_ESG_group = ['CTD', 'DHG', 'DPM', 'FPT', 'GAS', 'MBB']
low_ESG_group = ['NVL', 'PNJ', 'REE', 'SBT', 'SSI', 'STB', 'VIC', 'VNM']
non_ESG_group = ['CII', 'CTG', 'EIB', 'GMD', 'HDB', 'HPG', 'MSN', 'MWG', 'ROS', 'SAB', 'TCB', 'VCB', 'VHM', 'VJC', 'VPB', 'VRE']


In [55]:
companies = ['VCB']

# Calculate indicators

In [56]:
def calculate_indicators(df):
    if df.empty:
        return df
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['Bollinger_high'] = ta.volatility.bollinger_hband(df['close'], window=15, window_dev=2)
    df['Bollinger_low'] = ta.volatility.bollinger_lband(df['close'], window=15, window_dev=2)
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)

    return df

In [57]:
def macd_strategy(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI cross above 30 and MACD cross above Signal line
    df.loc[
        (df['close'] <= df['Bollinger_low']) &
        (df['RSI'] <= RSI_OVERSOLD), 'Signal'] = 1

    # Sell Signals: 
    df.loc[
        (df['RSI'] >= RSI_OVERBOUGHT) &
        (df['close'] >= df['Bollinger_high']), 'Signal'] = -1

    return df

In [58]:
def get_next_trading_day(date, trading_days):
    while date not in trading_days:
        date += BDay(1)
    return date

In [59]:
def calculate_monthly_rate(annual_rate, method='compounding'):
    """
    Converts an annual risk-free rate to a monthly rate.

    Parameters:
    annual_rate (float): The yearly risk-free rate as a decimal (e.g., 0.05 for 5%).
    method (str): The conversion method, either 'compounding' or 'simple'.

    Returns:
    float: The monthly risk-free rate.
    """
    if method == 'compounding':
        # Compounded monthly rate
        monthly_rate = (1 + annual_rate) ** (1 / 12) - 1
    elif method == 'simple':
        # Simple division approximation
        monthly_rate = annual_rate / 12
    else:
        raise ValueError("Method must be either 'compounding' or 'simple'")
    
    return monthly_rate

# Example usage
annual_rate = rf_2022  # For example, a 5% annual rate
rf_2022_monthly = calculate_monthly_rate(annual_rate, method='compounding')
monthly_rate_simple = calculate_monthly_rate(annual_rate, method='simple')

print(f"Monthly Rate (Compounding): {rf_2022_monthly:.6f}")
print(f"Monthly Rate (Simple): {monthly_rate_simple:.6f}")


Monthly Rate (Compounding): 0.002750
Monthly Rate (Simple): 0.002792


# Backtest

In [60]:
def simulate_investment(
    ticker, win_rate, loss_rate, mean_profit, mean_loss, 
    sell_fraction, start_date, end_date, f_star=1
):
    try:
        # Initialize trade counters and portfolio metrics
        number_of_buying_trades = 0
        number_of_selling_trades = 0
        cash = initial_investment
        holdings = 0
        portfolio_values = []

        # Load stock data and calculate indicators
        data = Vnstock().stock(symbol=ticker,source='TCBS').quote.history(start=start_date, end=end_date)
        data = data.set_index(pd.DatetimeIndex(data['time'].values))
        data = calculate_indicators(data)
        data = macd_strategy(data)

        trading_days = data.index
        buy_signals = data[data['Signal'] == 1].index
        sell_signals = data[data['Signal'] == -1].index

        pending_buy_shares = {}
        pending_sell_revenue = {}
        
        for i, current_date in enumerate(data.index):
            current_price = data['close'].iloc[i]

            # Handle pending T+2 settlements
            if current_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(current_date)
            if current_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(current_date)

            # Avoid trades in January 2024
            if current_date.month == 1 and current_date.year == 2024:
                portfolio_values.append(cash + holdings * current_price)
                continue

            # Buy if there's a buy signal and cash allows
            if current_date in buy_signals:
                allocation = cash * f_star
                shares_to_buy = int(allocation // current_price)
                total_cost = shares_to_buy * current_price
                if shares_to_buy > 0 and cash >= total_cost:
                    cash -= total_cost
                    settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                    pending_buy_shares[settlement_date] = pending_buy_shares.get(settlement_date, 0) + shares_to_buy
                    last_buy_price = current_price
                    number_of_buying_trades += 1

            # Sell if there's a sell signal and holdings allow
            if holdings > 0 and current_date in sell_signals:
                shares_to_sell = int(holdings * sell_fraction)
                revenue = shares_to_sell * current_price
                holdings -= shares_to_sell
                settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                pending_sell_revenue[settlement_date] = pending_sell_revenue.get(settlement_date, 0) + revenue
                number_of_selling_trades += 1

            # Update portfolio value
            portfolio_values.append(cash + holdings * current_price)

        # Finalize portfolio values including pending settlements
        final_date = data.index[-1]
        while final_date <= data.index[-1] + BDay(2):
            if final_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(final_date)
            if final_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(final_date)
            portfolio_values.append(cash + holdings * data['close'].iloc[-1])
            final_date += BDay(1)

        # Adjust portfolio values to match data index length
        if len(portfolio_values) > len(data.index):
            portfolio_values = portfolio_values[:len(data.index)]
        elif len(portfolio_values) < len(data.index):
            portfolio_values.extend([portfolio_values[-1]] * (len(data.index) - len(portfolio_values)))

        # Add portfolio values to data frame
        data['Portfolio_Value'] = portfolio_values
        data['Accumulated_Profit'] = data['Portfolio_Value'] - initial_investment
        data['Number_of_Buying_Trades'] = number_of_buying_trades
        data['Number_of_Selling_Trades'] = number_of_selling_trades
        data['Daily_Return'] = data['Portfolio_Value'].pct_change()

        # Calculate annual returns
        
        
        if start_date=='2022-01-01' and end_date=='2023-01-01':
            data['Month'] = data.index.month
            annual_data = data.groupby('Month').agg(
            Start_Value=('Portfolio_Value', 'first'),
            End_Value=('Portfolio_Value', 'last')
        )
        else:
            data['Year'] = data.index.year
            annual_data = data.groupby('Year').agg(
                Start_Value=('Portfolio_Value', 'first'),
                End_Value=('Portfolio_Value', 'last')
            )
        
        # Calculate Annual Return
        annual_data['Annual_Return'] = (annual_data['End_Value'] - annual_data['Start_Value']) / annual_data['Start_Value']
        
        # Merge back to main data frame if needed
        data = data.merge(annual_data[['Annual_Return']], left_on='Year' if 'Year' in data else 'Month', right_index=True, how='left')

        data['Cumulative_Max'] = data['Portfolio_Value'].cummax()  # Running max portfolio value
        data['Drawdown'] = (data['Portfolio_Value'] - data['Cumulative_Max']) / data['Cumulative_Max']
        max_drawdown = data['Drawdown'].min()  # Minimum drawdown value (most negative)

        data['Max_Drawdown'] = max_drawdown
        return data

    except Exception as e:
        print(f"Error occurred for {ticker}: {e}")
        return pd.DataFrame()


In [61]:
def calculate_sharpe_ratio(data, risk_free_rate=0.01):
    # Calculate daily returns from the Portfolio Value
    daily_returns = data['Annual_Return'].dropna()

    # Calculate average return and standard deviation of returns
    average_return = daily_returns.mean()
    std_deviation = daily_returns.std()

    # Calculate the Sharpe Ratio
    sharpe_ratio = (average_return - risk_free_rate) / std_deviation

    return sharpe_ratio

In [62]:
def calculate_sortino_ratio(data,rf=0.01):  # Target return can be set to risk-free rate
    # Calculate daily returns from the Portfolio Value
    daily_returns = data['Annual_Return'].dropna()

    # Calculate average return
    average_return = daily_returns.mean()

    # Calculate downside returns (returns below the target return)
    downside_returns = daily_returns[daily_returns < 0]

    # Calculate downside deviation
    downside_deviation = np.sqrt((downside_returns**2).mean()) if len(downside_returns) > 0 else np.nan

    # Calculate the Sortino Ratio
    sortino_ratio = (average_return - rf) / downside_deviation if downside_deviation > 0 else np.nan

    return sortino_ratio

thêm phần điều chỉnh f star ở code phía dưới

In [63]:
def backtest_multiple_companies(companies_vn30, win_rate, loss_rate, mean_profit, mean_loss, \
                                sell_fraction, start_date, end_date, rf):
    results = []
    for company in companies_vn30:
        result = simulate_investment(company, win_rate, loss_rate, mean_profit, mean_loss, \
                                     sell_fraction, start_date=start_date, end_date=end_date, f_star=f)
        if not result.empty:
            # Calculate the Sharpe Ratio for the result
            sharpe_ratio = calculate_sharpe_ratio(result,risk_free_rate=rf)
            results.append({
                'Company': company,
                'Final Portfolio Value': result['Portfolio_Value'].iloc[-1],
                'Total Profit': result['Accumulated_Profit'].iloc[-1],
                'Rate of Return': result['Accumulated_Profit'].iloc[-1] / initial_investment * 100,
                'Number of Buying Trades': result['Number_of_Buying_Trades'].iloc[-1],
                'Number of Selling Trades': result['Number_of_Selling_Trades'].iloc[-1],
                'Sharpe Ratio': sharpe_ratio,
                'Sortino Ratio': calculate_sortino_ratio(result,rf=rf),
                'MDD': result['Max_Drawdown'].min(),
            })
    return pd.DataFrame(results)

# Kelly 2019-2024

In [64]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
results_df = backtest_multiple_companies(high_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date='2019-01-01', end_date='2024-01-01', rf=rf_cal)
high_esg = pd.DataFrame(results_df)
print(high_esg)
average_rate_of_return = results_df['Rate of Return'].mean()
average_profit = results_df[results_df['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = results_df[results_df['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')

  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     CTD           1.305206e+08   30520619.22       30.520619   
1     DHG           1.011991e+08    1199117.13        1.199117   
2     DPM           1.088103e+08    8810258.58        8.810259   
3     FPT           1.121987e+08   12198732.79       12.198733   
4     GAS           1.118907e+08   11890655.98       11.890656   
5     MBB           1.000000e+08          0.00        0.000000   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                        1                         1     14.066533   
1                        2                         0    -27.053153   
2                        4                         1      9.922688   
3                        2                         1     11.540840   
4                        5                         1     11.431832   
5                        0                         0          -inf   

   Sortino Ratio       MDD  
0            NaN 

C:\Users\Admin\AppData\Local\Temp\ipykernel_5932\3161419043.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe_ratio = (average_return - risk_free_rate) / std_deviation


In [65]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
low_esg_kelly = backtest_multiple_companies(low_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date='2019-01-01', end_date='2024-01-04', rf=rf_cal)
low_esg_kelly = pd.DataFrame(low_esg_kelly)
print(low_esg_kelly)
average_rate_of_return = low_esg_kelly['Rate of Return'].mean()
average_profit = low_esg_kelly[low_esg_kelly['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = low_esg_kelly[low_esg_kelly['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')

C:\Users\Admin\AppData\Local\Temp\ipykernel_5932\3161419043.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe_ratio = (average_return - risk_free_rate) / std_deviation


  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     NVL           1.412180e+08   41217988.90       41.217989   
1     PNJ           1.068822e+08    6882238.73        6.882239   
2     REE           1.106614e+08   10661426.90       10.661427   
3     SBT           9.652729e+07   -3472711.70       -3.472712   
4     SSI           1.159421e+08   15942100.97       15.942101   
5     STB           1.000000e+08          0.00        0.000000   
6     VIC           1.058235e+08    5823506.05        5.823506   
7     VNM           1.040092e+08    4009220.23        4.009220   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                        3                         1      8.421254   
1                        5                         1      4.732665   
2                        4                         1      5.999271   
3                        1                         0    -15.708283   
4                        1                         0   

In [66]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
non_esg_kelly = backtest_multiple_companies(non_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date='2019-01-01', end_date='2024-01-04', rf=rf_cal)
non_esg_kelly = pd.DataFrame(non_esg_kelly)
print(non_esg_kelly)
average_rate_of_return = non_esg_kelly['Rate of Return'].mean()
average_profit = non_esg_kelly[non_esg_kelly['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = non_esg_kelly[non_esg_kelly['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')

C:\Users\Admin\AppData\Local\Temp\ipykernel_5932\3161419043.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe_ratio = (average_return - risk_free_rate) / std_deviation
C:\Users\Admin\AppData\Local\Temp\ipykernel_5932\3161419043.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe_ratio = (average_return - risk_free_rate) / std_deviation
C:\Users\Admin\AppData\Local\Temp\ipykernel_5932\3161419043.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe_ratio = (average_return - risk_free_rate) / std_deviation
C:\Users\Admin\AppData\Local\Temp\ipykernel_5932\3161419043.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe_ratio = (average_return - risk_free_rate) / std_deviation
C:\Users\Admin\AppData\Local\Temp\ipykernel_5932\3161419043.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe_ratio = (average_return - risk_free_rate) / std_deviation
C:\Users\Admin\AppData\Local\T

   Company  Final Portfolio Value  Total Profit  Rate of Return  \
0      CII           1.090988e+08    9098798.10        9.098798   
1      CTG           1.000000e+08          0.00        0.000000   
2      EIB           1.151013e+08   15101309.57       15.101310   
3      GMD           1.000000e+08          0.00        0.000000   
4      HDB           1.000000e+08          0.00        0.000000   
5      HPG           1.113473e+08   11347348.50       11.347349   
6      MSN           1.003768e+08     376784.70        0.376785   
7      MWG           1.066366e+08    6636598.14        6.636598   
8      ROS           1.000000e+08          0.00        0.000000   
9      SAB           1.081463e+08    8146282.87        8.146283   
10     TCB           1.099158e+08    9915795.22        9.915795   
11     VCB           1.085393e+08    8539323.32        8.539323   
12     VHM           9.678409e+07   -3215914.75       -3.215915   
13     VJC           1.000000e+08          0.00        0.00000

In [67]:
high_esg['ESG'] ='High'
low_esg_kelly['ESG'] = 'Low'
non_esg_kelly['ESG'] = 'Non'

In [68]:
merged = pd.concat([high_esg, low_esg_kelly, non_esg_kelly]) 
merged['period'] = '2019-2024' 

In [69]:
merged

,Company,Final Portfolio Value,Total Profit,Rate of Return,Number of Buying Trades,Number of Selling Trades,Sharpe Ratio,Sortino Ratio,MDD,ESG,period
0,CTD,1.305206e+08,30520619.22,30.520619,1,1,14.066533,NaN,-0.622270,High,2019-2024
1,DHG,1.011991e+08,1199117.13,1.199117,2,0,-27.053153,NaN,-0.772217,High,2019-2024
2,DPM,1.088103e+08,8810258.58,8.810259,4,1,9.922688,NaN,-0.784779,High,2019-2024
3,FPT,1.121987e+08,12198732.79,12.198733,2,1,11.540840,NaN,-0.535295,High,2019-2024
4,GAS,1.118907e+08,11890655.98,11.890656,5,1,11.431832,NaN,-0.952468,High,2019-2024
5,MBB,1.000000e+08,0.00,0.000000,0,0,-inf,NaN,0.000000,High,2019-2024
0,NVL,1.412180e+08,41217988.90,41.217989,3,1,8.421254,NaN,-0.804758,Low,2019-2024
1,PNJ,1.068822e+08,6882238.73,6.882239,5,1,4.732665,NaN,-0.976311,Low,2019-2024
2,REE,1.106614e+08,10661426.90,10.661427,4,1,5.999271,NaN,-0.772217,Low,2019-2024
3,SBT,9.652729e+07,-3472711.70,-3.472712,1,0,-15.708283,-1.794689,-0.522734,Low,2019-2024


# Kelly 2022

In [70]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
high_esg_2022 = backtest_multiple_companies(high_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date='2022-01-01', end_date='2023-01-01', rf=rf_2022_monthly)
high_esg_2022 = pd.DataFrame(high_esg_2022)
print(high_esg_2022)
average_rate_of_return = high_esg_2022['Rate of Return'].mean()
average_profit = high_esg_2022[high_esg_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = high_esg_2022[high_esg_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
high_esg_2022['ESG'] ='High'

C:\Users\Admin\AppData\Local\Temp\ipykernel_5932\3161419043.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe_ratio = (average_return - risk_free_rate) / std_deviation


  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     CTD           6.001679e+07  -39983207.79      -39.983208   
1     DHG           9.231716e+07   -7682844.25       -7.682844   
2     DPM           1.301966e+08   30196629.37       30.196629   
3     FPT           1.100056e+08   10005626.26       10.005626   
4     GAS           1.000000e+08          0.00        0.000000   
5     MBB           9.711030e+07   -2889695.09       -2.889695   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                       15                         1     -0.347698   
1                        5                         0     -0.385547   
2                        2                         1      0.440754   
3                        6                         1      0.112091   
4                        0                         0          -inf   
5                       10                         1      0.134484   

   Sortino Ratio       MDD  
0      -0.250517 

In [71]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
low_esg_kelly_2022 = backtest_multiple_companies(low_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date='2022-01-01', end_date='2023-01-01', rf=rf_2022_monthly)
low_esg_kelly_2022 = pd.DataFrame(low_esg_kelly_2022)
print(low_esg_kelly_2022)
average_rate_of_return = low_esg_kelly_2022['Rate of Return'].mean()
average_profit = low_esg_kelly_2022[low_esg_kelly_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = low_esg_kelly_2022[low_esg_kelly_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
low_esg_kelly_2022['ESG'] = 'Low'

C:\Users\Admin\AppData\Local\Temp\ipykernel_5932\3161419043.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe_ratio = (average_return - risk_free_rate) / std_deviation


  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     NVL           1.976364e+07  -80236359.80      -80.236360   
1     PNJ           1.096982e+08    9698178.80        9.698179   
2     REE           1.000000e+08          0.00        0.000000   
3     SBT           7.722407e+07  -22775928.01      -22.775928   
4     SSI           7.605039e+07  -23949611.61      -23.949612   
5     STB           8.057772e+07  -19422276.15      -19.422276   
6     VIC           7.591322e+07  -24086780.40      -24.086780   
7     VNM           1.007267e+08     726687.46        0.726687   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                       15                         1     -0.373760   
1                        4                         1      0.162973   
2                        0                         0          -inf   
3                       10                         1     -0.510295   
4                       14                         1   

In [72]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
non_esg_kelly_2022 = backtest_multiple_companies(non_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date='2022-01-01', end_date='2023-01-01', rf=rf_2022_monthly)
non_esg_kelly_2022 = pd.DataFrame(non_esg_kelly_2022)
print(non_esg_kelly_2022)
average_rate_of_return = non_esg_kelly_2022['Rate of Return'].mean()
average_profit = non_esg_kelly_2022[non_esg_kelly_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = non_esg_kelly_2022[non_esg_kelly_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
non_esg_kelly_2022['ESG'] = 'Non'

C:\Users\Admin\AppData\Local\Temp\ipykernel_5932\3161419043.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe_ratio = (average_return - risk_free_rate) / std_deviation


   Company  Final Portfolio Value  Total Profit  Rate of Return  \
0      CII           9.176722e+07   -8232776.06       -8.232776   
1      CTG           9.103602e+07   -8963983.39       -8.963983   
2      EIB           1.023148e+08    2314785.20        2.314785   
3      GMD           1.084103e+08    8410286.72        8.410287   
4      HDB           1.036989e+08    3698863.18        3.698863   
5      HPG           6.676728e+07  -33232724.55      -33.232725   
6      MSN           8.976808e+07  -10231919.67      -10.231920   
7      MWG           8.861303e+07  -11386968.58      -11.386969   
8      ROS           1.000000e+08          0.00        0.000000   
9      SAB           1.088029e+08    8802937.00        8.802937   
10     TCB           5.815813e+07  -41841866.45      -41.841866   
11     VCB           1.251191e+08   25119103.86       25.119104   
12     VHM           7.805170e+07  -21948297.07      -21.948297   
13     VJC           9.439056e+07   -5609441.20       -5.60944

In [73]:
merged_2022 = pd.concat([high_esg_2022, low_esg_kelly_2022, non_esg_kelly_2022])
merged_2022['period'] = '2022-2023'
merged_2022

,Company,Final Portfolio Value,Total Profit,Rate of Return,Number of Buying Trades,Number of Selling Trades,Sharpe Ratio,Sortino Ratio,MDD,ESG,period
0,CTD,6.001679e+07,-39983207.79,-39.983208,15,1,-0.347698,-0.250517,-0.948115,High,2022-2023
1,DHG,9.231716e+07,-7682844.25,-7.682844,5,0,-0.385547,-0.272037,-0.522734,High,2022-2023
2,DPM,1.301966e+08,30196629.37,30.196629,2,1,0.440754,0.610269,-0.616182,High,2022-2023
3,FPT,1.100056e+08,10005626.26,10.005626,6,1,0.112091,0.086359,-0.772217,High,2022-2023
4,GAS,1.000000e+08,0.00,0.000000,0,0,-inf,NaN,0.000000,High,2022-2023
5,MBB,9.711030e+07,-2889695.09,-2.889695,10,1,0.134484,0.138280,-0.953376,High,2022-2023
0,NVL,1.976364e+07,-80236359.80,-80.236360,15,1,-0.373760,-0.219112,-0.817825,Low,2022-2023
1,PNJ,1.096982e+08,9698178.80,9.698179,4,1,0.162973,0.215746,-0.951135,Low,2022-2023
2,REE,1.000000e+08,0.00,0.000000,0,0,-inf,NaN,0.000000,Low,2022-2023
3,SBT,7.722407e+07,-22775928.01,-22.775928,10,1,-0.510295,-0.340180,-0.999387,Low,2022-2023


# Gộp 2 cái

In [74]:
merged_all = pd.concat([merged, merged_2022])

In [75]:
merged_all= merged_all.reset_index()

In [76]:
merged_all

,index,Company,Final Portfolio Value,Total Profit,Rate of Return,Number of Buying Trades,Number of Selling Trades,Sharpe Ratio,Sortino Ratio,MDD,ESG,period
0,0,CTD,1.305206e+08,30520619.22,30.520619,1,1,14.066533,NaN,-0.622270,High,2019-2024
1,1,DHG,1.011991e+08,1199117.13,1.199117,2,0,-27.053153,NaN,-0.772217,High,2019-2024
2,2,DPM,1.088103e+08,8810258.58,8.810259,4,1,9.922688,NaN,-0.784779,High,2019-2024
3,3,FPT,1.121987e+08,12198732.79,12.198733,2,1,11.540840,NaN,-0.535295,High,2019-2024
4,4,GAS,1.118907e+08,11890655.98,11.890656,5,1,11.431832,NaN,-0.952468,High,2019-2024
5,5,MBB,1.000000e+08,0.00,0.000000,0,0,-inf,NaN,0.000000,High,2019-2024
6,0,NVL,1.412180e+08,41217988.90,41.217989,3,1,8.421254,NaN,-0.804758,Low,2019-2024
7,1,PNJ,1.068822e+08,6882238.73,6.882239,5,1,4.732665,NaN,-0.976311,Low,2019-2024
8,2,REE,1.106614e+08,10661426.90,10.661427,4,1,5.999271,NaN,-0.772217,Low,2019-2024
9,3,SBT,9.652729e+07,-3472711.70,-3.472712,1,0,-15.708283,-1.794689,-0.522734,Low,2019-2024


In [77]:
merged_all['total_trades']= merged_all['Number of Buying Trades'] + merged_all['Number of Selling Trades']
merged_all.to_csv('boll kelly.csv', index=False)

In [78]:
A= merged_all.groupby(['ESG','period'])[['Rate of Return', 'Sharpe Ratio', 'Sortino Ratio', 'total_trades', 'MDD']].mean()
A

Rate of Return  Sharpe Ratio  Sortino Ratio  total_trades  \
ESG  period                                                                 
High 2019-2024       10.769897          -inf            NaN         3.000   
     2022-2023       -1.725582          -inf       0.062471         7.000   
Low  2019-2024       10.132971          -inf      -1.794689         3.500   
     2022-2023      -20.005761          -inf      -0.186957         9.875   
Non  2019-2024        3.880639          -inf      -2.444341         2.750   
     2022-2023       -5.673144          -inf       0.144650         7.500   

                     MDD  
ESG  period               
High 2019-2024 -0.611172  
     2022-2023 -0.635437  
Low  2019-2024 -0.639855  
     2022-2023 -0.845445  
Non  2019-2024 -0.414468  
     2022-2023 -0.674279

In [79]:
merged_all.groupby(['ESG','period'])[['Rate of Return', 'Sharpe Ratio', 'Sortino Ratio', 'total_trades', 'MDD']].median()


Rate of Return  Sharpe Ratio  Sortino Ratio  total_trades  \
ESG  period                                                                 
High 2019-2024       10.350457     10.677260            NaN           2.5   
     2022-2023       -1.444848     -0.117803       0.086359           6.0   
Low  2019-2024        6.352872      4.623014      -1.794689           3.5   
     2022-2023      -21.099102     -0.359266      -0.244266          11.0   
Non  2019-2024        0.188392    -13.272202      -1.876705           2.0   
     2022-2023       -4.118795      0.024504      -0.023892           8.0   

                     MDD  
ESG  period               
High 2019-2024 -0.697244  
     2022-2023 -0.694199  
Low  2019-2024 -0.658322  
     2022-2023 -0.998057  
Non  2019-2024 -0.522734  
     2022-2023 -0.692673

In [84]:
kelly = pd.read_csv('boll kelly.csv')
no_kelly = pd.read_csv('no kelly.csv')

In [85]:
kelly['Kelly'] = 'Yes'
no_kelly['Kelly'] = 'No'
double_merged = pd.concat([kelly, no_kelly])
double_merged.to_excel('boll.xlsx', index=False)

In [86]:
B= double_merged.groupby(['ESG','period','Kelly'])[['Rate of Return', 'Sharpe Ratio', 'Sortino Ratio', 'total_trades', 'MDD']].mean()
B.to_excel('boll_double_merged.xlsx')